In [ ]:
#install dependencies/libraries
!pip install numpy==1.23.5
!pip install pandas==1.5.3
!pip install scikit-learn==1.2.1
!pip install tensorflow==2.12.0
!pip install keras==2.12.0

In [1]:
#Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Flatten, Reshape, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, regularizers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

In [3]:
# Read data from Excel CSV file
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/transformed_data.csv')
# Create DataFrame
df = pd.DataFrame(data)
# Prepare features and target
ftr = df.iloc[:,:-1] # Excludes target feature
target = df['DeepFake']  # Target Feature

In [4]:
# Splitting dataset to training and testing
ftr_train, ftr_test, target_train, target_test = train_test_split(ftr, target, test_size=0.2, random_state=42)

In [5]:
def make_discriminator_model(input_shape):
    model = Sequential([
        Flatten(input_shape=(input_shape,)),
        Dense(512),
        LeakyReLU(),
        Dropout(0.3),
        Dense(256),
        LeakyReLU(),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    return model

In [22]:
def make_discriminator_model(input_shape):
    model = Sequential([
        Flatten(input_shape=(input_shape,)),
        Dense(512, kernel_regularizer=regularizers.l2(0.01)),
        LeakyReLU(),
        Dropout(0.3),
        Dense(256, kernel_regularizer=regularizers.l2(0.01)),
        LeakyReLU(),
        Dropout(0.3),
        Dense(128, kernel_regularizer=regularizers.l2(0.01)),
        LeakyReLU(),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    return model

In [6]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(512))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(np.product([190]), activation='tanh'))
    model.add(layers.Reshape((190,)))
    return model

In [10]:
# Initialize models with correct input shapes
generator = make_generator_model()
discriminator = make_discriminator_model(ftr_train.shape[1])

<ipython-input-10-fb41223a8d5f>:2: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  generator = make_generator_model()


In [8]:
# Define the loss and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5)

In [11]:
# Training Loop
@tf.function
def train_step(images, batch_size):
    noise = tf.random.normal([batch_size, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [12]:
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch[0], tf.shape(image_batch[0])[0])

In [13]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [14]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.zeros_like(real_output), real_output)
    fake_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    return real_loss + fake_loss

In [23]:
BATCH_SIZE = 32
EPOCHS = 50
train_dataset = tf.data.Dataset.from_tensor_slices((ftr_train.values, target_train)).batch(BATCH_SIZE)
train(train_dataset, EPOCHS)

In [20]:
# Save generator
generator.save('/content/drive/MyDrive/Colab Notebooks/GAN_ML/Models/GAN_generator_model_extended_ftr_full.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
# Save discriminator
discriminator.save('/content/drive/MyDrive/Colab Notebooks/GAN_ML/Models/GAN_discriminator_model_extended_ftr_full.h5')

In [ ]:
# -- optional -- load pretrained generator and discriminator.
from keras.models import load_model
discriminator = load_model('/content/drive/MyDrive/Colab Notebooks/GAN_ML/Models/binary_discriminator_model_model_extended_ftr_full.h5')
generator = load_model('/content/drive/MyDrive/Colab Notebooks/GAN_ML/Models/binary_generator_model_extended_ftr_full.h5')

In [16]:
# Function to generate fake images for testing
def generate_fake_images(generator, num_images):
    noise = tf.random.normal([num_images, 100])
    generated_images = generator(noise, training=False)
    return generated_images.numpy()  # Convert tensors to numpy arrays for evaluation

In [17]:
def evaluate_discriminator(model, real_images, fake_images):
    # Concatenate real and fake images into one array
    images = np.concatenate([real_images, fake_images])
    # Correct label assignment: 1s for fake, 1s for real
    labels = np.concatenate([np.zeros(len(real_images)), np.ones(len(fake_images))])

    # Shuffle the data to mix real and fake images
    indices = np.arange(len(images))
    np.random.shuffle(indices)
    images = images[indices]
    labels = labels[indices]

    # Evaluate the model
    loss, accuracy = model.evaluate(images, labels, verbose=1)  # verbosity for detailed output
    print(f"Loss: {loss}, Accuracy: {accuracy * 100}%")


In [18]:
# Preparing real images from ftr_test_final for testing
real_images_test = ftr_test.to_numpy()

In [19]:
# Generate fake images for testing
num_test_images = real_images_test.shape[0]  # Generate as many fake images as there are real test images
fake_images_test = generate_fake_images(generator, num_test_images)

In [20]:
# Compile the discriminator model
discriminator.compile(
    optimizer=discriminator_optimizer,  # You can adjust the learning rate and other parameters as needed
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [21]:
# Evaluate the discriminator
evaluate_discriminator(discriminator, real_images_test, fake_images_test)

709/709 [==============================] - 2s 3ms/step - loss: 80.4562 - accuracy: 1.0000
Loss: 80.4561767578125, Accuracy: 100.0%
